**Imports**

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import zlib
import pickle
import os
import matplotlib.pyplot as plt
from itertools import combinations
import sys

compressed_sizes = {}


**Load dataset**

In [2]:
def load_dataset(file_path):
    df = pd.read_csv(file_path, sep="::", engine="python", names=["user_id", "item_id", "rating", "timestamp", "normalized_rating"])
    return df

**Dictionary with pre-computed values**

In [3]:
def compute_compressed_sizes(user_ratings):
    
    for user, ratings in user_ratings.items():
        u_string = "".join(f"{k}:{v}" for k, v in sorted(ratings.items()))
        compressed_sizes[u_string] = len(zlib.compress(u_string.encode()))
    
    return compressed_sizes


**Similarities**

In [11]:
def linear_similarity(ratings_u, ratings_v):
    common_items = set(ratings_u.keys()).intersection(set(ratings_v.keys()))
    if not common_items:
        return 0  # If no common items, similarity is 0

    diff_sum = sum(abs(ratings_u[i] - ratings_v[i]) for i in common_items)
    ls_value = 1 - (diff_sum / len(common_items))
    return max(0, ls_value)  # Ensure similarity is non-negative

# Compression Similarity with precomputed sizes
def compression_similarity(ratings_u, ratings_v):
    u_string = "".join(f"{k}:{v}" for k, v in sorted(ratings_u.items()))
    v_string = "".join(f"{k}:{v}" for k, v in sorted(ratings_v.items()))
    c_uv = len(zlib.compress((u_string + v_string).encode()))
    c_u = compressed_sizes[u_string]
    c_v = compressed_sizes[v_string]
    return 1 - (c_uv - min(c_u, c_v)) / max(c_u, c_v)

# Kolmogorov Similarity with precomputed sizes
def kolmogorov_similarity(ratings_u, ratings_v):
    u_string = "".join(f"{k}:{v}" for k, v in sorted(ratings_u.items()))
    v_string = "".join(f"{k}:{v}" for k, v in sorted(ratings_v.items()))
    c_u = compressed_sizes[u_string]
    c_v = compressed_sizes[v_string]
    return 1 / (1 + abs(c_u - c_v))


**Simlarity Matrix and graph**

In [13]:
# Compute user similarity matrix and construct graph (with precomputed sizes)
def compute_similarity_matrix(user_ratings, similarity_measure, compressed_sizes):
    similarity_graph = nx.Graph()
    for (u, v) in combinations(user_ratings.keys(), 2):
        sim = similarity_measure(user_ratings[u], user_ratings[v])
        if sim > 0.99:
            similarity_graph.add_edge(u, v, weight=sim)
    return similarity_graph

# Detect user clusters from similarity graph
def detect_groups(similarity_graph):
    return list(nx.connected_components(similarity_graph))

**Reputation-based intra-clustering**

In [6]:
def compute_cluster_ratings(df, user_groups):
    """ Compute initial ratings per cluster based only on the items rated by users in the cluster. """
    cluster_item_ratings = {}

    for cluster_idx, user_set in enumerate(user_groups, start=1):
        cluster_df = df[df["user_id"].isin(user_set)]
        item_avg_ratings = cluster_df.groupby("item_id")["normalized_rating"].mean().to_dict()
        cluster_item_ratings[cluster_idx] = item_avg_ratings
    
    return cluster_item_ratings

def fill_missing_ratings(cluster_item_ratings, total_items=20000):
    """Fill missing ratings for the largest clusters by borrowing from other clusters."""
    sorted_clusters = sorted(cluster_item_ratings.keys(), key=lambda c: len(cluster_item_ratings[c]), reverse=True)
    top_clusters = sorted_clusters[:3]
    
    for cluster in top_clusters:
        missing_items = set(range(1, total_items + 1)) - set(cluster_item_ratings[cluster].keys())
        for item in missing_items:
            for donor_cluster in sorted_clusters:
                if item in cluster_item_ratings[donor_cluster]:
                    cluster_item_ratings[cluster][item] = cluster_item_ratings[donor_cluster][item]
                    break
        cluster_item_ratings[cluster] = dict(sorted(cluster_item_ratings[cluster].items()))
        print(f"CLUSTER {cluster}: {len(cluster_item_ratings[cluster])} items rated", cluster_item_ratings[cluster])
    
    return cluster_item_ratings

def compute_reputation_adjusted_ratings(df, cluster_item_ratings, user_groups, lambda_factor=0.95, tol=1e-6):
    """ Iteratively adjust ratings based on user reputation until convergence. """
    sorted_clusters = sorted(cluster_item_ratings.keys(), key=lambda c: len(cluster_item_ratings[c]), reverse=True)
    top_clusters = sorted_clusters[:3]
    prev_ratings = {c: cluster_item_ratings[c].copy() for c in top_clusters}
    converged = False
    
    while not converged:
        new_ratings = {}
        converged = True  # Assume convergence until proven otherwise
        
        for cluster in top_clusters:
            cluster_users = user_groups[cluster - 1]  # Get users in this cluster
            cluster_df = df[df["user_id"].isin(cluster_users)]
            item_ratings = prev_ratings[cluster].copy()
            user_reputation = {}
            
            # Compute reputation scores
            for user in cluster_users:
                user_ratings = cluster_df[cluster_df["user_id"] == user]
                if len(user_ratings) == 0:
                    continue
                
                errors = [(user_ratings[user_ratings["item_id"] == item]["normalized_rating"].values[0] - item_ratings[item]) 
                          for item in user_ratings["item_id"] if item in item_ratings]
                avg_error = np.mean(errors) / len(user_ratings) if errors else 0
                user_reputation[user] = max(0, 1 - lambda_factor * avg_error)  # Ensure reputation is non-negative
            
            # Compute new weighted averages
            new_item_ratings = item_ratings.copy()  # Start with existing ratings
            for item in item_ratings.keys():
                ratings = cluster_df[cluster_df["item_id"] == item]
                if len(ratings) == 0:
                    continue
                
                weights = np.array([user_reputation[user] for user in ratings["user_id"] if user in user_reputation])
                values = np.array([ratings[ratings["user_id"] == user]["normalized_rating"].values[0] for user in ratings["user_id"] if user in user_reputation])
                
                if len(weights) > 0 and sum(weights) > 0:
                    new_item_ratings[item] = np.dot(weights, values) / sum(weights)
            
            new_ratings[cluster] = new_item_ratings
            
            # Check for convergence
            print("ITERATION")
            for item in new_item_ratings.keys():
                if abs(new_item_ratings[item] - prev_ratings[cluster].get(item, 0)) > tol:
                    converged = False
        
        prev_ratings = new_ratings.copy()
    print("new_ratings: ", new_ratings)
    return new_ratings

def calculate_item_averages(clusters):
    # Dictionary to store the aggregated items and their cluster appearances
    item_sums = {}
    item_counts = {}

    # Iterate through each cluster to gather data
    for cluster, items in clusters.items():
        for item, value in items.items():
            if item not in item_sums:
                item_sums[item] = 0
                item_counts[item] = 0
            item_sums[item] += value
            item_counts[item] += 1

    # Calculate the averages with one decimal place
    item_averages = {item: round(item_sums[item] / item_counts[item], 1) for item in item_sums}

    return item_averages




**Main function**

In [9]:

# Main execution
file_path = "/home/martim/Desktop/tese/datasets/ml-1m/normalized_ratings.dat"
df = load_dataset(file_path)

# Prepare user ratings
user_ratings = {user: dict(zip(group["item_id"], group["normalized_rating"])) for user, group in df.groupby("user_id")}

# Compute or load compressed sizes
compressed_sizes = compute_compressed_sizes(user_ratings)
print("comprimido")

comprimido


**Clustering**

In [16]:
# Choose similarity measure: linear_similarity, compression_similarity, kolmogorov_similarity
similarity_graph = compute_similarity_matrix(user_ratings, kolmogorov_similarity, compressed_sizes)

# Detect user clusters from similarity graph
user_groups = list(nx.connected_components(similarity_graph))

# Print the number of detected clusters
print(f"User groups: {(user_groups)}")

print("number of user_groups: ", len(user_groups))



KeyboardInterrupt: 

**Ranking computation**


In [15]:
# Assuming user_groups is already defined
cluster_item_ratings = compute_cluster_ratings(df, user_groups)  # Step 1: Compute initial ratings
    
# Fill missing ratings
filled_ratings = fill_missing_ratings(cluster_item_ratings)

cluster_item_ratings_reputation = compute_reputation_adjusted_ratings(df, filled_ratings, user_groups)

#calculate average on top3 clusters
item_averages = calculate_item_averages(cluster_item_ratings_reputation)
print("item_averages: ", item_averages)
print("len(item_averages): ", len(item_averages))

# Initialize a dictionary to count occurrences of each value between 0.1 and 1.0
value_counts = {round(i * 0.1, 1): 0 for i in range(1, 11)}

# Iterate through the item_averages dictionary and count the occurrences of each value
for value in item_averages.values():
    rounded_value = round(value, 1)
    if rounded_value in value_counts:
        value_counts[rounded_value] += 1

# Extract bins and counts
bins = list(value_counts.keys())
counts = list(value_counts.values())

# Define bin edges from 0.1 to 1.0 (inclusive)
bin_edges = np.arange(0.1, 1.1, 0.1)

# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.bar(bins, counts, width=0.08, align='edge', edgecolor='black')

# Adding numbers on top of bars
for i in range(len(counts)):
    plt.text(bins[i] + 0.05, counts[i] + 0.1, str(counts[i]), ha='center', fontsize=10)

# Set labels and title
plt.xlabel('Bin')
plt.ylabel('Number of Items')
plt.title('Kolmogorv Similarity | 0.8 similarity threshold | 0.95 reputation factor')

# Adjust the x-axis to display the correct bin edges
plt.xticks(bin_edges)

# Tight layout to avoid clipping
plt.tight_layout()

# Show the plot
plt.show()




CLUSTER 1: 3653 items rated {1: 0.8280228334198236, 2: 0.6466338259441707, 3: 0.6132029339853301, 4: 0.5510791366906475, 5: 0.6063241106719368, 6: 0.7816229116945107, 7: 0.6880407124681933, 8: 0.6072727272727273, 9: 0.5365853658536586, 10: 0.7135135135135134, 11: 0.7639308855291577, 12: 0.5016129032258064, 13: 0.654320987654321, 14: 0.6982456140350878, 15: 0.5152542372881356, 16: 0.761839863713799, 17: 0.8084880636604774, 18: 0.6743362831858407, 19: 0.4990825688073394, 20: 0.5104, 21: 0.7238636363636364, 22: 0.6787096774193548, 23: 0.5958762886597938, 24: 0.6373239436619719, 25: 0.7284571428571428, 26: 0.7125, 27: 0.5958333333333333, 28: 0.8197368421052632, 29: 0.8146478873239437, 30: 0.726984126984127, 31: 0.6455445544554456, 32: 0.7890270665691295, 33: 0.7, 34: 0.7779850746268657, 35: 0.6884615384615385, 36: 0.7915048543689321, 37: 0.6857142857142857, 38: 0.6363636363636364, 39: 0.7265905383360522, 40: 0.7760000000000001, 41: 0.8060301507537688, 42: 0.5771739130434783, 43: 0.69253731

KeyboardInterrupt: 